# Генерация текста с помощью LSTM рекуррентных нейронных сетей в Python с Keras



In [1]:
# Ипорт необходимых библиотек
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

## Подготовка даных

In [3]:
# Загрузка текста и приведение к нижнему регистру
filename = "tom_sawyer.txt"
raw_text = open(filename).read()
raw_text = raw_text.lower()

In [4]:
# Подготовка данных
# Создание словаря уникальных символов в числа
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [5]:
# Сведения о датасете
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  393551
Total Vocab:  51


Будем идти по тексту окном в 100 символов. Тогда каждый экземпляр входных данных будет состоять из 100 признаков X и иметь метку из одного символа Y - следующий, 101-й символ за ними. 

In [6]:
# подготовка данных на вход
seq_length = 100
dataX = []
dataY = []
for i in range(n_chars - seq_length):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[c] for c in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total training patterns: {}".format(n_patterns))

Total training patterns: 393451


## Трансформация данных в приемлимый вид для Keras и LSTM в частности

**Во-первых**, мы должны трансформировать наш список входных данных в форму, приемлимую для LSTM.  

Входные данные в LSTM долны быть трехмерные:  
**Образцы.** Одна последовательность - это один образец. Batch     принимает один или больше образцов.  
**Временные шаги.** Один временной шаг - это одна точка наблюдений в выборке.  
**Признаки.** Один признак - это наблюдение за временной шаге.

То есть LSTM ожидает на входе 3D-array c размерностями [samples, time steps, features]

**Во-вторых**, нам нужно привести числа в диапазон [0, 1], чтобы сделать образцы легче обучаемы для LSTM, использующей сигмоидную функцию активации по умолчанию.

In [7]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# нормализация к диапазону [0, 1]
X = X / float(n_vocab)

**В-третьих**, нам нужно конвертировать выходные метки в one-hot encoding. (т.е. все классы, которым не принадлежим 0, а для текущего класса 1)

In [8]:
y = np_utils.to_categorical(dataY)

## Определение LSTM модели

Мы определим один скрытый слой LSTM с 256 блоков памяти.  
Используем dropout с вероятностью 20%.  
Выходной слой - плотный слой с функцией автивации softmax, для выводы вероятности в диапазоне [0, 1] для каждого из символа словаря.  
Так как задача сводится к классификации по символам словаря, то функция потеря loss будет categorical_crossentropy, мы не используем classification accuracy, так как нам важна обобщенность, и мы не хотим переобучится.  
Алгоритм оптимизации ADAM для скорости.

In [9]:
# определение LSTM модели
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

Будем использовать checkpoint для сохранения весов сети, при улучшении loss. Будем использовать лучший набор весов для генераивной модели в будущем.

In [ ]:
# Определение checkpoint
filepath="weights-improvement-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

## Тренировка LSTM

Будем использовать 20 эпох и 128 экземпляров в batch.

In [ ]:
model.fit(X, y, epochs=50, batch_size=128, callbacks=callbacks_list)

Epoch 1/50
3071/3074 [============================>.] - ETA: 0s - loss: 2.0513
Epoch 00001: loss improved from 2.06380 to 2.05126, saving model to weights-improvement-01-2.0513.hdf5
3074/3074 [==============================] - 43s 14ms/step - loss: 2.0513
Epoch 2/50
3072/3074 [============================>.] - ETA: 0s - loss: 2.0393
Epoch 00002: loss improved from 2.05126 to 2.03933, saving model to weights-improvement-02-2.0393.hdf5
3074/3074 [==============================] - 43s 14ms/step - loss: 2.0393
Epoch 3/50
3073/3074 [============================>.] - ETA: 0s - loss: 2.0275
Epoch 00003: loss improved from 2.03933 to 2.02749, saving model to weights-improvement-03-2.0275.hdf5
3074/3074 [==============================] - 43s 14ms/step - loss: 2.0275
Epoch 4/50
3072/3074 [============================>.] - ETA: 0s - loss: 2.0173
Epoch 00004: loss improved from 2.02749 to 2.01731, saving model to weights-improvement-04-2.0173.hdf5
3074/3074 [==============================] - 43s 1

## Генерация текста с LSTM сетью

Загружаем веса с checkpoint

In [10]:
# загрузка весов
filename = "weights-improvement-50-1.8224.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

Создание обратного словаря из чисел в символы

In [11]:
int_to_char = dict((i, c) for i, c in enumerate(chars))

Самый простой способ генерировать последовательности текста с помощью LSTM это начать с произвольной последовательности, генерировать символ, прибавлять её к этой последовательности и отрезать у неё первый символ. Так можно сделать сколько угодно длинную сгенерированную последовательность.

In [14]:
# Выбор случайной последовательности
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed: {}".format(''.join([int_to_char[value] for value in pattern])))
# генерация символов
for i in range(1000): # генерируем последовательность из 1000 символов
	x = numpy.reshape(pattern, (1, len(pattern), 1))
	x = x / float(n_vocab)
	prediction = model.predict(x, verbose=0)
	index = numpy.argmax(prediction)
	result = int_to_char[index]
	seq_in = [int_to_char[value] for value in pattern]
	print(result, end='')
	pattern.append(index)
	pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed: ance, and shoutings and pistol-shots sent their
hollow reverberations to the ear down the sombre ais
eds of the siaee of the siee and she tiong thieg hes head and the sioes whth the seleoberen that was toon the seared toon the boys would never be aoneer thet was tomn the seared toon the boys and she pioete of the siee and the sioes and she pirete tas the siarehee that had been a groan aayenture of the seared toon the sioes and saed they were sooe tome the seared toon the siarehe of the sieee of the sieee of the siee and she sioe the seirol oo the siee and the sioe of the siarehen aadk the siarehen aedone the seared and the sioes sfat he was ao ond saake that he was an ond saake that he was ao ond saake that he was an ond saake that he was ao ond saake that he was an ond saake that he was ao ond saake that he was an ond saake that he was ao ond saake that he was an ond saake that he was ao ond saake that he was an ond saake that he was ao ond saake that he was an ond saake that he